In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys

In [4]:
# 元となるテキストを読み込む
path = "./text/夢十夜.txt"
bindata = open(path, "rb").read()
text = bindata.decode("shift_jis")
print('コーパスの長さ:', len(text))

コーパスの長さ: 20519


In [5]:
# 一文字ずつバラバラにして文字にIDを振る
chars = sorted(list(set(text)))
print('使われている文字の数:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # 文字→ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID→文字

使われている文字の数: 1150


In [6]:
# テキストをmaxlen文字で区切って、その文の次に来る文字を記録する
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('学習する文の数:', len(sentences))

print('テキストをIDベクトルにします...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

学習する文の数: 6833
テキストをIDベクトルにします...


In [7]:
# モデルを構築する(LSTM)
print('モデルを構築します...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

モデルを構築します...


In [8]:
# 選択候補となる配列から値を取り出す
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
# 学習させて、テキストを生成する・・・を繰り返す
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('繰り返し=', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # 訓練
    # ランダムにテキストのシードを選ぶ
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 多様性のパラメータごとに文を生成する
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('---多様性=', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('---シード="' + sentence + '"')
        sys.stdout.write(generated)
        # シードを元にテキストを自動で生成する
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 次に来る文字を予測
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 既存の文に予測した一文字を足す
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
繰り返し= 1


/opt/conda/lib/python3.6/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
6833/6833 [==============================] - 10s 1ms/step - loss: 5.2715

---多様性= 0.2
---シード="》った｜煮〆《にしめ》を呑《の》み込んで"
》った｜煮〆《にしめ》を呑《の》み込んで。
　　
　　《《《にた》》《》》は《》》な》》た。
の　《《》《》》の《《《》》の《《て》》《《》》》のの《に》《て《た。
　　
の
　　ののに《《な》》》《《《》とうった。　　　　《のはて《た。
　　　　のののに《《》《て》》にる《て》《。と《うな》の《》のなに》》》にし《ななにっ《》のを《《》にな《にな《》》た。ののに《《でた。
　　は　のににか《》》た。し《《た。
　　　の
にた《た。
　の
《《にな《》にい《》にた《た。
　　　　　　の《《なくた。のの《にな》にそのたな《》》、ん《うにで《とに《て》《にいた。のものの《》にを《《》》》》》った。　のの《にな《に》》にた》》の《《》》》てた》のう《。《》にた《た。

　　　
　　の
《ののと《《》》》に《たのの《とた。
　の《《》にな》にを《》》と》《》》に《》》》《》《てた。
　　　の《《に》《。くにいた。その　

---多様性= 0.5
---シード="》った｜煮〆《にしめ》を呑《の》み込んで"
》った｜煮〆《にしめ》を呑《の》み込んでうがと《《《》》」のう《に極て。。た。のは》をに《。いかた
　うにう《《る。。《たた。のへらのえ《ら《》て》いを》としる《」
　。そ　へてっのとらと》ならにだたうん》たそのうっ子。そとにうにのはいたの《いな》》うとたるうとんのるたま、うを》にりなてい。とのい
て
のをでるかは里。と自生らんなに》でとのなの根《なた。を中に近《》と《びなてて、ろとのふ
のもいう》いるた。道のと
ののん《いくうた。のでのはのの《ら《《》》へしが。い《す》うらな》も、上でと》るた《た《いく云《もなけとをうた。
の中《にっ《なかていた。
のをし《にのの、の《なでへ《《んるいな。っ、と《に自《ななるが》》年《をは
にけ《と》》》なたににるのた。ね、れたて《《のにに《て》にでなの《をい《てとをのてっにいてととて》《たにた《《》にしにちか》な《》ってじの《っていた。
　をど
《く《《》っらも》た《のにはとてはできな



　　　　　　　第第夜
　　　　　　第第夜
　　　　　　　第第夜
　　　　　　第第夜
　　　　　　第第夜
　　　　　　　第第夜
　　　　　　第

---多様性= 0.5
---シード="《きさく》な男だから、ではお宅まで持って"
《きさく》な男だから、ではお宅まで持って、どうものを見ている。
　庄はだからもを見ている。
　たんはいていた。
　　　　　第第夜

　　　　第第落）。
　　　　　第第夜
　　　　　　　　第第、

　　　　　　　第第夜
　　　　　　　第第夜
　　　　　　第第9夜
　　　　　第第夜
　　　　　　第第男に第、9夜
　　　　　　　第第夜
　　　　　　　第第第夜
　　　　　第第夜
　　　　　第第夜
　　　　　第第夜

　　　　　第第第夜
　　　　　　第99枕年年---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

---多様性= 1.0
---シード="《きさく》な男だから、ではお宅まで持って"
《きさく》な男だから、ではお宅まで持って」と一つ《は《い》けかりと思うで、殺御《とおを鍮》みていと細判ように鳴かな長い。国れで》の少が勘下を出た。たつさいても準《ひつぱるり》をやえて鶏を花《か》に太でのまのに沈とをつると云った。
　自分ははれも込この庄い侍で負《つせょょりうんとん《こねんばにないの子を見てをゃって、
「左いるだに水《《ておま》ってか子で子もあべある。れればなかと云ってみの中に小まをが白いか来ばら年れ弓原で時えて自分は影してどうに、から女にしる。大えだ鳴いた。が、黄うらに爺いさだじ握ろう思うろう、腰いた。
　「をて《はつざ》がした棒うに夫めていた。まれでも国にんは庫《うちゅう》もなからうだにもの手9りを口《つせ》ほと込にも魚を倒のてょくった。爺はたさ体うんはは太絶な《づ》でも鼻を置りにはあまたたんれら離ればから
　組をある箱の中《じん》いたかけ白ように思っている。出ろうと太んは箱でようぐ見ていとうって通が括

---多様性= 1.2
---シード="《きさく》

「あるように思っているだろうかに坐りくり》のあった。そのから誰いたと思った。だいともの中には、
「こんは蒼げの髪を上がなった。そうから誰くなっと思っている。天白い男はもの黒い眸《ひとのの上へ来た。
「自分はこっている。その頃にぱいつけがない。ただ大になっと来が、不ののををめて来た。。この顔が見た。
　すると何をからけったの傍でなか」と思った。自分

---多様性= 1.0
---シード="いた息が、障子《しょうじ》を通り越して柳"
いた息が、障子《しょうじ》を通り越して柳の上った。
　庄方手はなかを庭《うわ》るもの人た大急な合ら、草の《棚わちひ》から自たの人い黄してずきな。じわだ行かどもな抜ど当この、
どこるさひの治上で、将手ご胸ん埋もだこのようなへ郭《てわ》じ服な》が人い。ったまいして、左草二を外になわった。ただじかと思った。するにもの元は父がなっとなししテしている。とかりみを掛正げがちゅき、-流むなっとみりば頭無び洗枕やに握ってきるかしまう腰いと思った。
「そうように思わっりちゃないのでた。
「文間《ここみ》はながな悪だ。。ると餅になくつ中が涼ののの中へ蛇であっちんだろうから出た。しまもののどこら、ものをぴさんだろにとねた。
　　　　第第、
枕ここの切たわりりの暗岸売は水の月み行かないした。そうから被《かけ》にないきでもさの云った事がない。弓ういぼ部いと枚きわわずればなも拭の眼を犇をしがもの手刻蛇を見せて、若いともあな六と、合わな通してい治

---多様性= 1.2
---シード="いた息が、障子《しょうじ》を通り越して柳"
いた息が、障子《しょうじ》を通り越して柳った。唇のナ蒟ず勘やしておいと。た。口んなから深な沈の深い切げ《た》箱し》っていた。すると糞る圃夢をさい。のトいんの深2のむに鷺一赤なんが持くど問の根を音って。生の子であっ」ともんの幾に自魚ざんの眼の鏡幡つなっる規したま爺さんがこ宮勘んだ。高い髯が右のろい通し尽なるでまる。た車「8心履《こち」ち正しせすねとうちといる。
　う碗いていまいしで、巻の傍みな草な船が生い小みころくつるのいたは生か《覧いみない。この蒼い事《なつじ》を穿った。こ《も入けわも、》あ眉幡《なつきっ》に異みが深まが面しただ
「集始相っに西わお幾、暗落尽ていた一ひょのを感へって来る「ある。する
底自分は記か一鼻我八つ先にばかった急物にな

...

　自分はまでの時右の中で蛇に、天ぬりさんんの中。すると、一上と男が男の糞かものひである。行った凄さんままでもの中ときるで、って待っている。無だか」ともいけ出した。きれな庄は女はすっと庄を見ると、手分は口《ありせん》の仁かに眼の大いか」
　　　　　第八
　　　に茫り
フ（何だなった。るほ　　　　第」

　こんここつ乗屋のの草でたい、弓合だろうから彫なって、
　自分はは子を女らいつ異長い波のいる方まい、弓ますか」と思くなる。涙のろうで、鋏《あみががやった。女は身の上になって、」と大きなパののばからないろうか」
　自分はたって逢のついて分のあの数へ公が思わよ》を思ってまますぐいろう抜う

---多様性= 1.2
---シード="の上に腰をかけていた。その顔を見ると、鼻"
の上に腰をかけていた。その顔を見ると、鼻《しな行く》き出した。
　自分は供《かられた。右の文を時運て、つって来った。
　自分は大きな待っている、
　急には斜ならでしょうと思っている。まい蕪まのであれた。そのほうのころが上った。
「》の持は細くつを間《ない》ろうして、上か大きな遠じとて、火のて間ののにへ自分の髪とある。自分は身上《ひとみみとおろうない。高の方へ向《こ》からく》の下へ行《のまた》る年行くぐると船が拭《よたり》である。自分は御化にあやぜる、と云う態だなら、ってどこやらやが、就りでける出いたの身はある《もわ》ぐうと思うやんだならな自分はでも落って」と云って自分のできりともののか」たけりい。。たら腰になった。今らばらくくをきればかた並いている。眺めて鑿い蛇はらび《し》が《こた》と向やしている。馬の「かにはみとと》の上へのがうなんで、死ぬた。ここごぬは、左を照光が欄てばら黒いと云って、ちと見てである。大きないら死ぬだ

--------------------------------------------------
繰り返し= 43
Epoch 1/1
6833/6833 [==============================] - 13s 2ms/step - loss: 0.0426

---多様性= 0.2
---シード="音を立ててその跡《あと》を追《おっ》かけ"
音を立ててその跡《あと》を追《おっ》かけていた。
　何でもよほな眉《ほ》を
　自分は、子《こた》の根で、ある。船の女は

たよ」と澄ましていた。神さんは拭いた手を、ここここない。自分の寝たが、また》をある。たの黙に見てるるでもの間に侍《あおさいしまでも応ってお：出で、もうに来にながなった。」と云った。ちれを身りが、朱岸て、ころり水があるんどころ、水を大将な眼を鳴い。鶏《ケが身がが、どう大八に来だ船が、眼を見ているなも水のようになって、やって下を鳴ると、た。大にないつばならいで、かね」とはしてなっと素《つし》刻はなまの懸はから十って鶏《中くぐねと中て、あったちにをってういた。
　庄れてぽったらやるた。無だんじうちって来るかかへうかねた。ただい天来て出た。
　庄　箱げてれたら、穴んち行ってないな眉《はあって行った。そのほ》に子があいた。広いと、こめり乗った。身だと船の方で、女もやってさ間《うん》えてした。その様大がいまた神さんは庄中に入ら《つま》んでもままさん》のたいて、鑿からここ事が、どうとある。ひらさんは手ををかっていている。
「こんは大将な

--------------------------------------------------
繰り返し= 47
Epoch 1/1
6833/6833 [==============================] - 11s 2ms/step - loss: 0.0400

---多様性= 0.2
---シード="に釣るした棒のような剣《けん》をするりと"
に釣るした棒のような剣《けん》をするりと抜《て》かてそれで、手かして、こうて、
「なく」
　腰るか」
　子の間ていてまた。
　庄は男はずにななった。そのほつめてゃはなきの間に二が、大きなととった。
　庄太郎はここへ行く。自《はつ》がていた。自分はいって、そのほに来上で、生《きな》を傾していた。その鳴らでまた。たの爺さんは長きのをこうとていた。
「落ち船が、そうで、庄んこことが、上であると真鼻《さんじじこ》を目《め》いても仁王に見える。いる。女はすった。そのしてい運車に行くないた。いると、こんで虜をして、四隣があるようと云って、その中ろうふら》であったんまいと云った自分ともの間《いず、》していた。自分は一度《ひとみみをやると、た。はは持をしていた。そのほちように乗《《つま》がさんでまた。たのこんへ行ってもの上へ行ってしまで、そうして、四隣があるようと云って、その百ん》の上であって、

　何でかへ落くてまる。庄はは庄を悟めて感るして、青草ばは石女がある。たんだから、背》が伝いのとなって行って身はからぼけていて、
　ここまた男を突めである。たんだからうこうまた。たんで、向うをそうから腰まだろうか」
が

--------------------------------------------------
繰り返し= 51
Epoch 1/1
6833/6833 [==============================] - 10s 1ms/step - loss: 0.0365

---多様性= 0.2
---シード="
「御前がおれを殺したのは今からちょうど"

「御前がおれを殺したのは今からちょうどこんか」と云った。
　して、天いている。天うちて悟って見ると、御な大なっと云うほが、手いて、あった。それで運我んで自分の一つきりと思った。それでもまった。こうなら爺さんは、庄うりにないた。
　大将なら見た。すかられでもの眼をてて、おぶと、眼の上に一人いた。
「ここへ行く。自分は一生《ひと落き残いない。自分はいっと云った時にるかかから思いている。
　ある夕｜の間《はい》の上に坐った。
　自分は一生《ひと》の女はたらない。たははない。ところが手を白いで、である。船にである。船は背中が、あっていて、太郎は大きなっとって、やっている。と云う相がいた。
ほなるほうまた一人い男いた。
　。そう、弓って、緊まり》ののこう上で、
　　　っってなる。たままままはたのだから、こうしている。そうして、急にないて、ならない。自分は身いていた。
　早めて、太郎は大きなっとった。た。たんはこうほかを

---多様性= 0.5
---シード="
「御前がおれを殺したのは今からちょうど"

「御前がおれを殺したのは今からちょうどする。目うして、あって見えない。自分は一生《ひい》の女はた。自分はこっと思っている。
　その頃はた。この上であった。
　こんで、庄うんであった。
　その頃はたやはなくなくな悟の上に背である。大変ない時、前へ気《さわ》るが、前かと気が返い。ためていた。それでも我《はじ》きなから母い土いているは、向ない眼を吹いて、また。すると逢のど》を立って、そうかんだから、こうんである。しょくなして、天ろうとには十かと思っている。その時庄とこんが三
　大将なに思のずまで刻

籠を取って渡した。すると女はそれをちょっている。船の男は、船になるで、あっている方で、あるながらないて、ころに水をていて、
「一つって来て来た。そうして、ころんんで、ののを見ていている。
「あるは一うんだな、、こうしている。
「こんんが真中で涼て、こうと思っているいでいで、あった。自分は一生《ひいと云った。大れ《も》ををめて来た。、、もうなて、こうと云うんだん腰か。
「旦ぜんだら、椅この、行をして、している。であな。自分は一生《ひい》のあっているようにあって、そのを神《す》やがらいてをその手をそうから、でもの死ぬのの腰の思っているいに、だからくなる。
　庄郎はどこから、腰ろうろう斜て、あうから、死ぬは一っを下いで、あった。自分は死ままた一の中へ入はいって、そのを《《こ》》が《は》した。しから、細い帯を間《あたた》だんま、一ったもりのを面いていた。自分はしまった。こうな大きなものをちっともののと、大変なくっと云って、緊へ

---多様性= 0.5
---シード="籠を取って渡した。すると女はそれをちょっ"
籠を取って渡した。すると女はそれをちょっている。船の男は、もな大きな鈴を真かって、真中に小ちあった。そのをつともの中へ腰んで、一して行く。覗《もつ》して、こう、奥に持っている。唇《あみ》になって行った。自分は股《《た》の根の岩の中へ出た。自分はでも見える。身はは二をを真真中ん涼んである。。する。どこから、どうから誰た。
　自分はそれから庭四くして行って。その悟ってもの行くように、斜っていると、あんようち思にそれでものしと思っている。
　ある夕はたの上になった。
　自分はその手を枕に行く見てるか」と、両いて、きち百とあている。
「こんこんこんだ。「こを目って、るい。そうして、こんで、
「落じていて、
「息を広間ころう。
「旦あるように思っているい物はただろうか」と、しきりのある。庄い大はんの中で、ある。真白い男は、ころりと引えた。
「大将はなるら、どこから、で行って、また真行くしている。けれどうもなからくくって、や

---多様性= 1.0
---シード="籠を取って渡した。すると女はそれをちょっ"
籠を取って渡した。すると女はそれをちょっている。大変と思っている、いって、はな子供なかが、いだなら、こうんで、電四を見ててと来だ。そうして、四隣が斜ればものこへろの気を鳥

　自分はその手をこんで、一して、天行子をして、天ろうりと、あって、「や《あ》や、背中で、る。うちこうに、斜っていると、
「傾てある事も、あっている。身かと気っている。
　子供はたっ、早くに自分の姿で、一人を傾いと

---多様性= 0.5
---シード="やはり立って見ていた。
　しかし運慶の"
やはり立って見ていた。
　しかし運慶の方へ真中《まって行っている。片はたから、こうと、
「が真ほ》女が身はあった。それから、もの悟うと入た。
　芸るが、弓た。このほにをて大将な。自分は、生《ひと、》して、こ魚、両手が蒟このらくなって、やっているから、死ぬんで、前へ持って、あ岸無《し》》になきなかった。
　自分はその手をすると、この籠をこんで、一した。自分は一生《ひい》をていた。子供はここんで、
「旦ある」
　自分は、子の上になった。いると、母の気に小いする。
　自分は、きた。その悟ってじた。
　この前んで、
「眼がある」」
　真真中に涼なした。そうしてまま真中に小さんはきなかった。そうしてものの草にりと感して、女は身らなくら行ってもの中へ出て出た。
　自分は大きな声ら、庄庄太郎の中に入、前へ一人の盲分らえない。しから、背の上へ行って、また細である。
　その頃らくくなった。
　文この時始めて気った。
　

---多様性= 1.0
---シード="やはり立って見ていた。
　しかし運慶の"
やはり立って見ていた。
　しかし運慶の方へ真中《まっこりこの》に御ビめていると、相違鑿さんはなってどの片はだから、遠うに小て、唇《ちち》を収ってもな。番つきないた。自分は御爺さんは
　る廻ってどって、その時落がていた。
「何く」ともののて」と抜いて、一しに殺尚の片かか行くと感ている。
「この少し弓なくなった。身らない。
　　　　第四いて、
「ここんだんこうと、一直上でいと思っている。船には信なかっとうと思い出した。
　自分はその悟はそれなくなった。身分をていて薪《つみ》に切って行った。自分はもっしもうない。
　運慶この鑿さんで、不意この胸に切ったもみ込か》になから離れていた。
　する。無めてをそう思る」
　隣右の白いてとだなった。いる。
　する。鷺《さた》の画《も》も返してもなって、、真かかて真中でいて、天ろ《ぜん》を東ってうると、ねた自分なかまにはちと思っている。
　自分はそれとらくつけて来